In [ ]:
#pip install faiss-cpu
#pip install beautifulSoup4
import os
from dotenv import load_dotenv
#from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
#example of prompt template
from langchain_core.prompts import ChatPromptTemplate
#converts the chat model output into a string for convenience
from langchain_core.output_parsers import StrOutputParser
#we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore
from langchain_openai import OpenAIEmbeddings
#import FAISS as vectorstore:
from langchain_community.vectorstores import FAISS
#import text splitter to break up text within the scraped URL:
from langchain_text_splitters import RecursiveCharacterTextSplitter
#import and use WebBaseLoader for retrival example
from langchain_community.document_loaders import WebBaseLoader
#create retrieval chain: 
from langchain.chains.combine_documents import create_stuff_documents_chain
#use the retriever to dynamically select the most relevant documents and pass those in for a given question
from langchain.chains import create_retrieval_chain

#set_debug(True)

# Load environment variables from .env file
load_dotenv()

# Access the API keys
openai_api_key = os.getenv('OPENAI_API_KEY')
#langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
#langsmith = os.getenv('LANGCHAIN_TRACING_V2')

# Initialize the chat model with the API key
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model="gpt-3.5-turbo",
    temperature=0
)

#example of invoking API with a prompt
#llm.invoke("how can langsmith help with testing?")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an interviewer for the company Dataiku."),
    ("user", "{input}")
])

output_parser = StrOutputParser()

#create your chain
chain = prompt | llm | output_parser

#invoke your chain
chain.invoke({"input": "how can I best prepare for my sales engineer interview at dataiku? Only give a 2 sentence response."})

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

embeddings = OpenAIEmbeddings()

#Now, we can use this embedding model to ingest documents into a vectorstore. 
#We will use a simple local vectorstore, FAISS, for simplicity's sake


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])